In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/FastaiNotebook_02_fully_connected")' FastaiNotebook_02_fully_connected

Installing packages:
	.package(path: "/content/FastaiNotebook_02_fully_connected")
		FastaiNotebook_02_fully_connected
With SwiftPM flags: []
Working in: /tmp/tmpqeao4gw5/swift-install
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/saeta/Just
Fetching https://github.com/latenitesoft/NotebookExport
Completed resolution in 3.05s
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.3
Cloning https://github.com/latenitesoft/NotebookExport
Resolving https://github.com/latenitesoft/NotebookExport at 0.6.0
Cloning https://github.com/saeta/Just
Resolving https://github.com/saeta/Just at 0.7.3
[1/11] Compiling Path Path+StringConvertibles.swift
[2/11] Compiling Path Path+ls.swift
[3/11] Compiling Path Path->Bool.swift
[4/11] Compiling Path Path.swift
[5/11] Compiling Path Extensions.swift
[6/11] Compiling Path Path+Attributes.swift
[7/11] Compiling Path Path+Codable.swift
[8/11] Compiling Path Path+CommonDirectories.swift
[9/11

In [0]:
//export
import Foundation
import TensorFlow
import Path

In [0]:
import FastaiNotebook_02_fully_connected

Does nn.Conv2d init work well?¶

In [4]:
var (xTrain, yTrain, xValid, yValid) = loadMNIST(path: Path.home/".fastai"/"data"/"mnist_tst")
let (trainMean, trainStd) = (xTrain.mean(), xTrain.standardDeviation())
xTrain = normalize(xTrain, mean: trainMean, std: trainStd)
xValid = normalize(xValid, mean: trainMean, std: trainStd)

In [5]:
xTrain = xTrain.reshaped(to: [xTrain.shape[0], 28, 28, 1])
xValid = xValid.reshaped(to: [xValid.shape[0], 28, 28, 1])
print(xTrain.shape, xValid.shape)

[60000, 28, 28, 1] [10000, 28, 28, 1]


In [0]:
let images = xTrain.shape[0]
let classes = xValid.max() + 1
let channels = 32

In [0]:
var layer1 = FAConv2D<Float>(filterShape: (5, 5, 1, channels)) //Conv2D(1, nh, 5)

In [0]:
let x = xValid[0..<100]

In [12]:
x.shape

▿ [100, 28, 28, 1]
  ▿ dimensions : 4 elements
    - 0 : 100
    - 1 : 28
    - 2 : 28
    - 3 : 1


In [0]:
extension Tensor where Scalar: TensorFlowFloatingPoint {
    func stats() -> (mean: Tensor, std: Tensor) {
        return (mean: mean(), std: standardDeviation())
    }
}

In [14]:
(filter: layer1.filter.stats(), bias: layer1.bias.stats())

▿ 2 elements
  ▿ filter : 2 elements
    - mean : -0.0007354039
    - std : 0.201281
  ▿ bias : 2 elements
    - mean : 0.0
    - std : 0.0


In [0]:
withDevice(.cpu){
  let result = layer1(x)
}

In [0]:
let result = layer1(x)

In [17]:
result.stats()

▿ 2 elements
  - mean : 0.002124782
  - std : 0.90949047




This is in 1a now so this code is disabled from here:

var rng = PhiloxRandomNumberGenerator.global

extension Tensor where Scalar: TensorFlowFloatingPoint {

    init(kaimingNormal shape: TensorShape, negativeSlope: Double = 1.0) 
    {
        // Assumes Leaky ReLU nonlinearity
        let gain = Scalar(sqrt(2.0 / (1.0 + pow(negativeSlope, 2))))
        let spatialDimCount = shape.count - 2
        let receptiveField = shape[0..<spatialDimCount].contiguousSize
        let fanIn = shape[shape.count - 2] * receptiveField
        self.init(randomNormal: shape,
                  stddev: gain / sqrt(Scalar(fanIn)),
                  generator: &rng
        )
    }
}



In [18]:
layer1.filter = Tensor(kaimingNormal: layer1.filter.shape, negativeSlope: 1.0)
layer1(x).stats()

▿ 2 elements
  - mean : 0.0037424834
  - std : 1.0423291


In [0]:
// export
func leakyRelu<T: TensorFlowFloatingPoint>(
    _ x: Tensor<T>,
    negativeSlope: Double = 0.0
) -> Tensor<T> {
    return max(0, x) + T(negativeSlope) * min(0, x)
}

In [20]:
layer1.filter = Tensor(kaimingNormal: layer1.filter.shape, negativeSlope: 0.0)
leakyRelu(layer1(x)).stats()

▿ 2 elements
  - mean : 0.41658816
  - std : 0.79095083


In [21]:
var layer1 = FAConv2D<Float>(filterShape: (5, 5, 1, channels)) //Conv2D(1, nh, 5)
leakyRelu(layer1(x)).stats()

▿ 2 elements
  - mean : 0.28651994
  - std : 0.5065314


In [22]:
layer1.filter.shape

▿ [5, 5, 1, 32]
  ▿ dimensions : 4 elements
    - 0 : 5
    - 1 : 5
    - 2 : 1
    - 3 : 32


In [23]:
let spatialDimCount = layer1.filter.rank - 2
let receptiveField = layer1.filter.shape[0..<spatialDimCount].contiguousSize
receptiveField

25


In [24]:
let filtersIn = layer1.filter.shape[2]
let filtersOut = layer1.filter.shape[3]
print(filtersIn, filtersOut)

1 32


In [25]:
let fanIn = filtersIn * receptiveField
let fanOut = filtersOut * receptiveField
print(fanIn, fanOut)

25 800


In [0]:
func gain(_ negativeSlope: Double) -> Double {
    return sqrt(2.0 / (1.0 + pow(negativeSlope, 2.0)))
}

In [27]:
(gain(1.0), gain(0.0), gain(0.01), gain(0.1), gain(sqrt(5.0)))

▿ 5 elements
  - .0 : 1.0
  - .1 : 1.4142135623730951
  - .2 : 1.4141428569978354
  - .3 : 1.4071950894605838
  - .4 : 0.5773502691896257


In [28]:
(2 * Tensor<Float>(randomUniform: [10000]) - 1).standardDeviation()

0.57589847


In [0]:
//export
extension Tensor where Scalar: TensorFlowFloatingPoint {
    init(kaimingUniform shape: TensorShape, negativeSlope: Double = 1.0) {
        // Assumes Leaky ReLU nonlinearity
        let gain = Scalar.init(TensorFlow.sqrt(2.0 / (1.0 + TensorFlow.pow(negativeSlope, 2))))
        let spatialDimCount = shape.count - 2
        let receptiveField = shape[0..<spatialDimCount].contiguousSize
        let fanIn = shape[shape.count - 2] * receptiveField
        let bound = TensorFlow.sqrt(Scalar(3.0)) * gain / TensorFlow.sqrt(Scalar(fanIn))
        self = bound * (2 * Tensor(randomUniform: shape, generator: &PhiloxRandomNumberGenerator.global) - 1)
    }
}

In [30]:
layer1.filter = Tensor(kaimingUniform: layer1.filter.shape, negativeSlope: 0.0)
leakyRelu(layer1(x)).stats()

▿ 2 elements
  - mean : 0.4240648
  - std : 0.8359458


In [31]:
layer1.filter = Tensor(kaimingUniform: layer1.filter.shape, negativeSlope: sqrt(5.0))
leakyRelu(layer1(x)).stats()

▿ 2 elements
  - mean : 0.1911112
  - std : 0.3638833


In [0]:
public struct Model: Layer {
    public var conv1 = FAConv2D<Float>(
        filterShape: (5, 5, 1, 8),   strides: (2, 2), padding: .same, activation: relu
    )
    public var conv2 = FAConv2D<Float>(
        filterShape: (3, 3, 8, 16),  strides: (2, 2), padding: .same, activation: relu
    )
    public var conv3 = FAConv2D<Float>(
        filterShape: (3, 3, 16, 32), strides: (2, 2), padding: .same, activation: relu
    )
    public var conv4 = FAConv2D<Float>(
        filterShape: (3, 3, 32, 1),  strides: (2, 2), padding: .valid
    )
    public var flatten = Flatten<Float>()

    @differentiable
    public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: conv1, conv2, conv3, conv4, flatten)
    }
}

In [0]:
let y = Tensor<Float>(yValid[0..<100])
var model = Model()

In [34]:
let prediction = model(x)
prediction.stats()

▿ 2 elements
  - mean : -0.13231449
  - std : 0.1049914


In [35]:
let gradients = gradient(at: model) { model in
    meanSquaredError(predicted: model(x), expected: y)
}

gradients.conv1.filter.stats()

▿ 2 elements
  - mean : 0.10806452
  - std : 0.23033115


In [0]:
for keyPath in [\Model.conv1, \Model.conv2, \Model.conv3, \Model.conv4] {
    model[keyPath: keyPath].filter = Tensor(kaimingUniform: model[keyPath: keyPath].filter.shape)
}

In [38]:
let prediction = model(x)
prediction.stats()

▿ 2 elements
  - mean : -0.018766165
  - std : 0.1850172


In [39]:
let gradients = gradient(at: model) { model in
    meanSquaredError(predicted: model(x), expected: y)
}

gradients.conv1.filter.stats()

▿ 2 elements
  - mean : 0.059680127
  - std : 0.31526127


Export

In [40]:
import NotebookExport
let exporter = NotebookExport(Path.cwd/"02a_why_sqrt5.ipynb")
print(exporter.export(usingPrefix: "FastaiNotebook_"))

success
